In [1]:
import pandas as pd

In [2]:
df_ROSL=pd.read_csv('../datasets/preprocessedROS.csv')
df_ROSL

,Unnamed: 0,Location of herniation,ODI,입원기간,통증기간(월),수술시간,Seg Angle(raw),수술기법
0,0,3,51.000000,2,1.0,68.0,14.4,0.0
1,1,4,26.000000,1,1.0,31.0,17.8,0.0
2,2,3,32.188357,1,1.0,78.0,10.2,0.0
3,3,4,18.000000,1,2.0,73.0,19.9,0.0
4,4,4,32.540831,2,1.0,29.0,5.2,0.0
...,...,...,...,...,...,...,...,...
1889,1889,1,29.022749,2,12.0,80.0,21.8,0.0
1890,1890,2,30.308698,4,6.0,20.0,5.0,0.0
1891,1891,3,30.639320,3,1.0,50.0,17.4,1.0
1892,1892,2,27.895562,4,24.0,25.0,17.7,0.0


In [3]:
# df_ROSL = df_ROS[['Location of herniation','ODI', '입원기간', '통증기간(월)', '수술시간', '수술기법', 'Seg Angle(raw)' ]]
# df_ROSL[:2]

In [4]:
df_ROSL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1894 entries, 0 to 1893
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              1894 non-null   int64  
 1   Location of herniation  1894 non-null   int64  
 2   ODI                     1894 non-null   float64
 3   입원기간                    1894 non-null   int64  
 4   통증기간(월)                 1894 non-null   float64
 5   수술시간                    1894 non-null   float64
 6   Seg Angle(raw)          1894 non-null   float64
 7   수술기법                    1894 non-null   float64
dtypes: float64(5), int64(3)
memory usage: 118.5 KB


In [5]:
df_ROSL.isnull().sum()

Unnamed: 0                0
Location of herniation    0
ODI                       0
입원기간                      0
통증기간(월)                   0
수술시간                      0
Seg Angle(raw)            0
수술기법                      0
dtype: int64

In [6]:
df_ROSL_dropna = df_ROSL.dropna()
df_ROSL_dropna.isnull().sum()
df_ROSL_dropna.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1894 entries, 0 to 1893
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              1894 non-null   int64  
 1   Location of herniation  1894 non-null   int64  
 2   ODI                     1894 non-null   float64
 3   입원기간                    1894 non-null   int64  
 4   통증기간(월)                 1894 non-null   float64
 5   수술시간                    1894 non-null   float64
 6   Seg Angle(raw)          1894 non-null   float64
 7   수술기법                    1894 non-null   float64
dtypes: float64(5), int64(3)
memory usage: 118.5 KB


#### learning
- raw data를 target_train_split를 나눠서 모델 학습 및 GridSearchCV를 하고 평가를 한다. 
- 설명변수의 범주형: 수술기법을 제외하고 모델학습시킨다. 

In [7]:
df_ROSL_dropna.columns

Index(['Unnamed: 0', 'Location of herniation', 'ODI', '입원기간', '통증기간(월)',
       '수술시간', 'Seg Angle(raw)', '수술기법'],
      dtype='object')

In [8]:
target = df_ROSL_dropna['Location of herniation']
features = df_ROSL_dropna[['ODI','입원기간','통증기간(월)','수술시간','수술기법','Seg Angle(raw)']]
target.shape, features.shape

((1894,), (1894, 6))

In [9]:
from sklearn.model_selection import train_test_split
features_train, features_test, target_train, target_test = train_test_split(features, target, random_state=111)
features_train.shape, features_test.shape, target_train.shape, target_test.shape

((1420, 6), (474, 6), (1420,), (474,))

In [10]:
from sklearn.tree import DecisionTreeClassifier

In [11]:
model = DecisionTreeClassifier()

In [12]:
from sklearn.model_selection import GridSearchCV

In [13]:
hyper_params = {'min_samples_leaf' : [2, 9], 
                'max_depth' : [2, 9], 
                'min_samples_split' : [2, 9]}

#### 평가 score : 분류-정확도, 예측-R squre

In [14]:
from sklearn.metrics import f1_score

In [15]:
grid_search = GridSearchCV(model, param_grid = hyper_params, cv=3, verbose=1)

In [16]:
grid_search.fit(features_train, target_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [2, 9], 'min_samples_leaf': [2, 9],
                         'min_samples_split': [2, 9]},
             verbose=1)

In [17]:
grid_search.best_estimator_

DecisionTreeClassifier(max_depth=9, min_samples_leaf=2)

In [18]:
grid_search.best_score_, grid_search.best_params_

(0.5542516718554399,
 {'max_depth': 9, 'min_samples_leaf': 2, 'min_samples_split': 2})

In [19]:
best_model = grid_search.best_estimator_
best_model

DecisionTreeClassifier(max_depth=9, min_samples_leaf=2)

In [20]:
target_test_predict = best_model.predict(features_test)
target_test_predict

array([3, 3, 3, 3, 2, 2, 3, 1, 3, 2, 2, 3, 1, 3, 2, 2, 2, 3, 4, 1, 3, 2,
       3, 1, 1, 3, 2, 3, 3, 1, 1, 1, 1, 3, 1, 2, 2, 3, 3, 3, 1, 2, 4, 2,
       2, 3, 4, 4, 3, 2, 2, 3, 3, 2, 2, 3, 2, 3, 2, 3, 1, 2, 1, 3, 1, 3,
       2, 4, 1, 3, 1, 3, 2, 2, 2, 2, 3, 1, 1, 1, 1, 1, 3, 1, 1, 3, 3, 2,
       3, 1, 3, 1, 1, 1, 2, 2, 2, 2, 2, 4, 4, 1, 2, 2, 1, 2, 3, 2, 3, 3,
       3, 2, 3, 3, 2, 3, 1, 1, 3, 2, 1, 3, 1, 3, 3, 1, 1, 1, 1, 1, 2, 2,
       1, 3, 3, 3, 1, 3, 2, 2, 1, 3, 1, 3, 2, 3, 4, 2, 3, 3, 2, 2, 2, 2,
       2, 2, 2, 2, 4, 2, 2, 3, 3, 3, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 3, 2,
       2, 2, 1, 2, 1, 1, 1, 3, 2, 3, 2, 1, 3, 1, 3, 1, 3, 3, 4, 2, 1, 2,
       3, 3, 1, 3, 2, 3, 3, 1, 3, 3, 3, 2, 3, 1, 3, 1, 3, 1, 1, 4, 3, 2,
       2, 3, 3, 3, 1, 4, 3, 2, 3, 3, 1, 1, 3, 1, 3, 3, 2, 2, 3, 3, 3, 3,
       3, 3, 1, 1, 1, 1, 3, 2, 3, 1, 3, 2, 1, 1, 2, 2, 4, 2, 3, 3, 2, 2,
       2, 3, 2, 1, 1, 4, 1, 2, 1, 1, 3, 2, 2, 3, 1, 1, 1, 3, 2, 3, 3, 1,
       2, 2, 2, 1, 1, 2, 2, 2, 3, 1, 2, 2, 2, 2, 3,

In [21]:
from sklearn.metrics import classification_report

In [22]:
print(classification_report(target_test, target_test_predict))

              precision    recall  f1-score   support

           1       0.67      0.67      0.67       125
           2       0.62      0.60      0.61       156
           3       0.63      0.71      0.67       154
           4       0.36      0.30      0.33        30
           5       0.00      0.00      0.00         9

    accuracy                           0.62       474
   macro avg       0.46      0.46      0.46       474
weighted avg       0.61      0.62      0.62       474



C:\Users\01-15\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\01-15\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\01-15\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
import pickle

In [24]:
with open ('../datasets/RecurrenceOfSurgery_model.pkl', 'wb') as pkl_model :
    pickle.dump(obj=best_model, file=pkl_model)
    pass

In [25]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

In [26]:
confusion_matrix(target_test, target_test_predict)

array([[ 84,  21,  18,   2,   0],
       [ 28,  94,  30,   4,   0],
       [  8,  29, 109,   8,   0],
       [  4,   5,  12,   9,   0],
       [  1,   3,   3,   2,   0]], dtype=int64)

### best model 찾기

In [27]:
from collections import Counter
# sampling 하기전
Counter(target)

Counter({3: 657, 4: 147, 2: 563, 5: 34, 1: 493})

In [28]:
##under sampling_ 많은쪽을 끌어내림.
from imblearn.under_sampling import NearMiss

In [29]:
sampling_auto = {
1: auto, 2: auto, 3: auto, 4: auto, 5: auto
}

NameError: name 'auto' is not defined

In [ ]:
resample = NearMiss(sampling_strategy=sampling_auto)

In [ ]:
features_under_sample, target_under_resample = resample.fit_resample(features, target)

In [ ]:
from sklearn.metrics import make_scorer, f1_score
scoring = make_scorer(f1_score)
hyper_list = {'max_depth':range(2, 10),
              'min_samples_leaf': range(2, 10),
              'criterion': ['gini','entropy'],
              'class_weight': [None, 'balanced'],
              'min_samples_split': range(2, 10)}
# hyper_list = {'model__max_depth':range(2, 3),}

grid_model = GridSearchCV(model, param_grid=hyper_list,
                          scoring=scoring,
                          n_jobs = -1,
                          cv = 5)

In [ ]:
Counter(target)

### UnderSampling